# Trabajo Práctico 1:  Algoritmo de Maximización de la Esperanza

**Profesor**: Saúl Calderón Ramírez

**Estudiantes**:

* Ricardo Chacon
* Marco Ferraro
* Gabriel Valentine


In [ ]:
import pandas as pd

df = pd.read_csv('./data/pima-indians-diabetes.csv', header=None)

df.head()

In [ ]:


# TODO: #2 Fix variables names in graph functions
import matplotlib.pyplot as plt


def graph_histogram(df, column, target_column, column_tag="Name"):
  
  target_0 = df[df[target_column] == 0]
  target_1 = df[df[target_column] == 1]

  plt.hist(target_0[column], bins=20, alpha=0.5, label='Target 0', color='blue')
  plt.hist(target_1[column], bins=20, alpha=0.5, label='Target 1', color='red')

  plt.xlabel('Data Column')
  plt.ylabel('Frequency')
  plt.legend()
  plt.title('Histograms for Target 0 and Target 1')
  plt.show()

  return None

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm

def graph_normal_curve(df, column, target_column, column_tag="Name"):
    target_0 = df[df[target_column] == 0]
    target_1 = df[df[target_column] == 1]

    mu_0, std_0 = norm.fit(target_0[column])
    mu_1, std_1 = norm.fit(target_1[column])

    x = np.linspace(min(df[column]) - 3 * std_0, max(df[column]) + 3 * std_1, 100)
    pdf_0 = norm.pdf(x, mu_0, std_0)
    pdf_1 = norm.pdf(x, mu_1, std_1)

    plt.plot(x, pdf_0, label='Target 0 (Normal Curve)', color='blue')
    plt.plot(x, pdf_1, label='Target 1 (Normal Curve)', color='red')

    plt.fill_between(x, 0, pdf_0, alpha=0.2, color='blue')
    plt.fill_between(x, 0, pdf_1, alpha=0.2, color='red')

    plt.xlabel(column_tag)
    plt.ylabel('Probability Density')
    plt.legend()
    plt.title(f'Filled Normal Curves for Target 0 and Target 1 ({column_tag})')
    plt.show()

In [ ]:
graph_histogram(df, 1, 8)

In [ ]:
graph_normal_curve(df, 1, 8)

In [ ]:

# TODO: #3 Function to set up distance between density function